In [28]:
import polars as pl
import tempfile
import zipfile
from pathlib import Path
import geopandas as gpd

In [29]:
actual = pl.read_parquet(
    "gs://crea-pm25ml-examples/combined__omi_full_nn.parquet"
)

In [30]:
with tempfile.TemporaryDirectory() as tmpdir:
    zip_path = Path("../../grid_india_10km_shapefiles.zip")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(tmpdir)
        grid_path = Path(tmpdir, "grid_india_10km", "grid_india_10km.shp")
        
        grid = gpd.read_file(grid_path)
        # Convert grid_id to int and then to string
        grid["grid_id"] = grid["grid_id"].astype(int).astype(str)


FileNotFoundError: [Errno 2] No such file or directory: '../../grid_india_10km_shapefiles.zip'

In [ ]:
expected_rows = float(actual.shape[0])

In [ ]:
# Count the number of nulls in each column and plot the results where the number of nulls is greater than 0
null_counts_df = actual.select(
    [pl.col(col).is_null().sum().alias(col) for col in actual.columns]
).unpivot(
    variable_name="column",
    value_name="null_count"
).filter(
    pl.col("null_count") > 0
).with_columns(
    missing_percent=pl.col("null_count") / expected_rows * 100.0
).sort("missing_percent", descending=True)

In [ ]:
null_counts_df.plot.bar(
    x="column",
    y="missing_percent",
)

alt.Chart(...)

It looks like the columns that have null values are:
 - ERA5 - which we're already handling spatially
 - and the columns which we're going to use ML for